In [3]:
import cv2 as cv
import va
from ipywidgets import interact
import numpy as np

In [4]:
num_frames = 20
background_pixels = np.zeros((486, 644, 3), dtype=np.float32)
bg_list = []
for camera_i in range(1, 5):
    background_pixels = np.zeros((486, 644, 3), dtype=np.float32)
    with np.load(f'./data/cam{camera_i}/config.npz') as file:
        camera_matrix, dist_coeffs, rvec_extr, tvec_extr, R = [file[i] for i in [
            'camera_matrix', 'dist_coeffs', 'rvec_extr', 'tvec_extr', 'R']]
        cap = cv.VideoCapture(f"./data/cam{camera_i}/background.avi")
        for n_frame in range(num_frames):
            retF, frame = cap.read()
            if retF:
                height, width, channels = frame.shape
                for y in range(0, height):
                    for x in range(0, width):
                        background_pixels[y, x] += frame[y, x]

        cap.release()

        background_pixels = np.array(background_pixels / num_frames, dtype=np.uint8)
        bg_list.append(background_pixels)


In [11]:
@interact(camId=(1, 4), h_tresh1=(0, 256), s_tresh1=(0, 256), v_tresh1=(0, 256), h_tresh2=(0, 256), s_tresh2=(0, 256), v_tresh2=(0, 256))
def test_openclose(camId=1, h_tresh1=0, h_tresh2=0, s_tresh1=0, s_tresh2=0, v_tresh1=0, v_tresh2=0):
    path = f"./data/cam{camId}/video.avi"
    cap = cv.VideoCapture(path)
    ret, frame = cap.read()
    cap.release()
    frame_hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
    w, h, _ = frame.shape
    mask = np.zeros((w, h), dtype=np.uint8)
    background_pixels_hsv = cv.cvtColor(background_pixels, cv.COLOR_BGR2HSV)
    foreground_hsv = cv.absdiff(frame_hsv, background_pixels_hsv)
    thresh = 30
    # cv.imshow('foreground_gray', foreground_hsv)
    # cv.waitKey(0)
    # cv.destroyAllWindows()
    #foreground_hsv = cv.inRange(foreground_hsv, (0, 0, thresh), (180, 100, 100))
    # _, mask = cv.threshold(foreground_gray,thresh,255,cv.THRESH_BINARY)
    for x in range(foreground_hsv.shape[0]):
        for y in range(foreground_hsv.shape[1]):
            if foreground_hsv[x, y, 0] in range(h_tresh1, h_tresh2) and foreground_hsv[x, y, 1] in range(s_tresh1, s_tresh2) and foreground_hsv[x, y, 2] in range(v_tresh1, v_tresh2):
                mask[x, y] = 255

    contours, hierarchy = cv.findContours(
        image=mask, mode=cv.RETR_TREE, method=cv.CHAIN_APPROX_NONE)
    frame_copy = frame.copy()
    cv.drawContours(image=frame_copy, contours=contours, contourIdx=-1,
                    color=(0, 255, 0), thickness=2, lineType=cv.LINE_AA)

    va.show(frame_copy)


interactive(children=(IntSlider(value=1, description='camId', max=4, min=1), IntSlider(value=0, description='h…